In [ ]:
# Текстурные характеристики:
#  - Реализуйте подсчет гистограммы 
#  - Реализуйте рассчет текстурных характеристик Laws 
#  - Реализуйте расчет текстурных характеристик на основе матрицы взаимной встречаемости GLCM
# Для каждого случая: выполните тренировку классификатора (knn, svm, trees)
# Решите задачу сегментации данных на основе классификации изображений каждым методом.
# Выполните ручную разметку изображений на регионы.
# Сравните результат полученной классификации и размеченной.

In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import graycomatrix, graycoprops
from tqdm import tqdm

# загрузка изображений

def load_dataset(dataset_path):
    data = []
    labels = []
    imagePaths = []

    print("Загрузка изображений и извлечение признаков...")
    for root, dirs, files in os.walk(dataset_path):
        for f in files:
            if f.endswith(".png") or f.endswith(".jpg"):
                path = os.path.join(root, f)
                label = os.path.basename(os.path.dirname(path))
                imagePaths.append(path)
                labels.append(label)

    print(f"Изображений: {len(imagePaths)}")
    return imagePaths, labels

# признаки

def extract_histogram_features(image):
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def extract_glcm_features(image):
    glcm = graycomatrix(image, [5], [0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    return np.array([contrast, dissimilarity, homogeneity, energy, correlation])

def extract_laws_features(image):
    L5 = np.array([1, 4, 6, 4, 1])
    E5 = np.array([-1, -2, 0, 2, 1])
    S5 = np.array([-1, 0, 2, 0, -1])
    R5 = np.array([1, -4, 6, -4, 1])

    filters = [np.outer(f1, f2) for f1 in [L5, E5, S5, R5] for f2 in [L5, E5, S5, R5]]
    features = []

    for f in filters:
        filtered = cv2.filter2D(image, -1, f)
        features.append(np.mean(np.abs(filtered)))
    return np.array(features)

# Обучение и проверка

def evaluate_features(X, y, name):
    print(f"\n{name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    models = {
        "kNN": KNeighborsClassifier(n_neighbors=3),
        "SVM": SVC(kernel="linear"),
        "DecisionTree": DecisionTreeClassifier(random_state=42)
    }

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        print(f"\n-- {model_name} -- Accuracy: {acc:.3f}")
        print(classification_report(y_test, preds))

    return model  

# выбор регионов

def select_regions_and_classify(image_path, model, feature_func):
    print(f"\nИспользуется изображение: {image_path}")
    true_label = os.path.basename(os.path.dirname(image_path))
    print(f"Истинный класс: {true_label}")

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_display = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    regions = []
    start_point = None
    end_point = None
    drawing = False

    def draw_rectangle(event, x, y, flags, param):
        nonlocal start_point, end_point, drawing, img_display
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            start_point = (x, y)
            end_point = (x, y)
        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                end_point = (x, y)
                temp_img = img_display.copy()
                cv2.rectangle(temp_img, start_point, end_point, (0, 0, 255), 2)
                cv2.imshow("Выберите регионы (ENTER -> закончить, ESC -> закрыть)", temp_img)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            end_point = (x, y)
            regions.append((start_point, end_point))
            cv2.rectangle(img_display, start_point, end_point, (0, 0, 255), 2)
            cv2.imshow("Выберите регионы (ENTER -> закончить, ESC -> закрыть)", img_display)

    cv2.namedWindow("Выберите регионы (ENTER -> закончить, ESC -> закрыть)")
    cv2.setMouseCallback("Выберите регионы (ENTER -> закончить, ESC -> закрыть)", draw_rectangle)

    print("Выберите регионы вручную. Нажмите ENTER после выбора всех регионов.")
    print("ESC для закрытия окна без выбора.")

    while True:
        key = cv2.waitKey(1) & 0xFF
        cv2.imshow("Выберите регионы (ENTER -> закончить, ESC -> закрыть)", img_display)
        if key == 13:  # ENTER
            break
        elif key == 27:  # ESC
            regions = []  # отмена выбора
            break

    cv2.destroyAllWindows()

    if not regions:
        print("Нет выбранных регионов.")
        return

    print("\nРезультаты классификации выделенных регионов:")
    for i, ((x1, y1), (x2, y2)) in enumerate(regions):
        x1, x2 = max(0, min(x1, x2)), min(img.shape[1], max(x1, x2))
        y1, y2 = max(0, min(y1, y2)), min(img.shape[0], max(y1, y2))
        patch = img[y1:y2, x1:x2]

        if patch.size == 0:
            print(f"Регион {i+1} пустой, пропущен.")
            continue

        feat = feature_func(patch).reshape(1, -1)
        pred = model.predict(feat)[0]
        print(f"Регион {i+1}: предсказано -> {pred}")

        cv2.putText(img_display, pred, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        cv2.rectangle(img_display, (x1, y1), (x2, y2), (0,255,0), 2)

    cv2.imwrite("selected_regions_result.jpg", img_display)
    print("Сохранено изображение с отмеченными регионами -> selected_regions_result.jpg")

if __name__ == "__main__":
    dataset_path = "./kth_tips_grey_200x200/KTH_TIPS"
    imagePaths, labels = load_dataset(dataset_path)

    X_hist = []
    X_glcm = []
    X_laws = []

    for path in tqdm(imagePaths, desc="Обработка изображений"):
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        X_hist.append(extract_histogram_features(img))
        X_glcm.append(extract_glcm_features(img))
        X_laws.append(extract_laws_features(img))

    X_hist, X_glcm, X_laws = np.array(X_hist), np.array(X_glcm), np.array(X_laws)
    y = np.array(labels)

    model_hist = evaluate_features(X_hist, y, "Histogram")
    model_glcm = evaluate_features(X_glcm, y, "GLCM")
    model_laws = evaluate_features(X_laws, y, "Laws")

    print("\nОбучение завершено ✅")

    test_image_path = imagePaths[100]
    select_regions_and_classify(test_image_path, model_laws, extract_laws_features)

